In [1]:
from importlib.metadata import version

print("torch version: ", version("torch"))
print("tiktoken version : ", version("tiktoken"))

torch version:  2.1.1
tiktoken version :  0.3.3


In [2]:
!pip install torch

DEPRECATION: Loading egg at c:\python311\lib\site-packages\vboxapi-1.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
version("torch")

'2.1.1'

Tokenizing text


In [4]:
import os
import urllib.request




In [5]:
with open("text.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [6]:
len(raw_text)

20479

In [7]:
raw_text

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no great surprise to me to hear that, in the height of his glory, he had dropped his painting, married a rich widow, and established himself in a villa on the Riviera. (Though I rather thought it would have been Rome or Florence.)\n\n"The height of his glory"--that was what the women called it. I can hear Mrs. Gideon Thwing--his last Chicago sitter--deploring his unaccountable abdication. "Of course it\'s going to send the value of my picture \'way up; but I don\'t think of that, Mr. Rickham--the loss to Arrt is all I think of." The word, on Mrs. Thwing\'s lips, multiplied its _rs_ as though they were reflected in an endless vista of mirrors. And it was not only the Mrs. Thwings who mourned. Had not the exquisite Hermia Croft, at the last Grafton Gallery show, stopped me before Gisburn\'s "Moon-dancers" to say, with tears in her eyes: "We shall not look upon its like again"?\n\nWell!--even 

In [8]:
type(raw_text)

str

In [9]:
raw_text[:100]

'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no g'

The pattern r'(\s)' is a regular expression:
    
   * r before the string indicates a raw    string, meaning backslashes are treated literally.

   * (\s) matches any whitespace character (like spaces, tabs, or newlines). The parentheses around \s create a capturing group, which means the matched whitespace characters will be included in the resulting list.

In [11]:
import re

text= "Hello, world. My name is Mukund Agarwalla"
result= re.split(r'(\s)', text)

result

['Hello,',
 ' ',
 'world.',
 ' ',
 'My',
 ' ',
 'name',
 ' ',
 'is',
 ' ',
 'Mukund',
 ' ',
 'Agarwalla']

r before the string denotes a raw string, meaning backslashes are treated literally and not as escape characters.
   
   * ([,.]|\s) is the regular expression pattern used for splitting the text.
    
   * [,.] matches any comma , or period . character. The square brackets [ ] define a character class that matches any one of the characters inside them.

   * \s matches any whitespace character (spaces, tabs, newlines, etc.).

   * | is the logical OR operator in regular expressions, meaning it matches either the pattern on its left or the pattern on its right.

   * () around [,.]|\s create a capturing group, meaning the matched delimiters (commas, periods, and whitespaces) will be included in the resulting list.

In [17]:
result= re.split(r'([,.]|\s)', text)

result

['Hello',
 ',',
 '',
 ' ',
 'world',
 '.',
 '',
 ' ',
 'My',
 ' ',
 'name',
 ' ',
 'is',
 ' ',
 'Mukund',
 ' ',
 'Agarwalla']

In [18]:
result= [item for item in result if item.split()]
print(result)

['Hello', ',', 'world', '.', 'My', 'name', 'is', 'Mukund', 'Agarwalla']


In [21]:
preprocessed= re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
preprocessed= [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [22]:
len(preprocessed)

4690

In [23]:
all_words= sorted(set(preprocessed))

In [26]:
vocab_size= len(all_words)
vocab_size

1130

   * enumerate is a built-in function that takes an iterable (in this case, all_words) and returns an iterator of tuples, where each tuple contains an index and the corresponding item from the iterable.
   * For example, if all_words is ['apple', 'banana', 'cherry'], enumerate(all_words) produces:

[(0, 'apple'), (1, 'banana'), (2, 'cherry')]

In [27]:
vocab= {token:integer for token,integer in enumerate(all_words)}

In [28]:
vocab

{0: '!',
 1: '"',
 2: "'",
 3: '(',
 4: ')',
 5: ',',
 6: '--',
 7: '.',
 8: ':',
 9: ';',
 10: '?',
 11: 'A',
 12: 'Ah',
 13: 'Among',
 14: 'And',
 15: 'Are',
 16: 'Arrt',
 17: 'As',
 18: 'At',
 19: 'Be',
 20: 'Begin',
 21: 'Burlington',
 22: 'But',
 23: 'By',
 24: 'Carlo',
 25: 'Chicago',
 26: 'Claude',
 27: 'Come',
 28: 'Croft',
 29: 'Destroyed',
 30: 'Devonshire',
 31: 'Don',
 32: 'Dubarry',
 33: 'Emperors',
 34: 'Florence',
 35: 'For',
 36: 'Gallery',
 37: 'Gideon',
 38: 'Gisburn',
 39: 'Gisburns',
 40: 'Grafton',
 41: 'Greek',
 42: 'Grindle',
 43: 'Grindles',
 44: 'HAD',
 45: 'Had',
 46: 'Hang',
 47: 'Has',
 48: 'He',
 49: 'Her',
 50: 'Hermia',
 51: 'His',
 52: 'How',
 53: 'I',
 54: 'If',
 55: 'In',
 56: 'It',
 57: 'Jack',
 58: 'Jove',
 59: 'Just',
 60: 'Lord',
 61: 'Made',
 62: 'Miss',
 63: 'Money',
 64: 'Monte',
 65: 'Moon-dancers',
 66: 'Mr',
 67: 'Mrs',
 68: 'My',
 69: 'Never',
 70: 'No',
 71: 'Now',
 72: 'Nutley',
 73: 'Of',
 74: 'Oh',
 75: 'On',
 76: 'Once',
 77: 'Only',
 7

In [29]:
type(vocab)

dict

In [34]:
vocab.items()

dict_items([(0, '!'), (1, '"'), (2, "'"), (3, '('), (4, ')'), (5, ','), (6, '--'), (7, '.'), (8, ':'), (9, ';'), (10, '?'), (11, 'A'), (12, 'Ah'), (13, 'Among'), (14, 'And'), (15, 'Are'), (16, 'Arrt'), (17, 'As'), (18, 'At'), (19, 'Be'), (20, 'Begin'), (21, 'Burlington'), (22, 'But'), (23, 'By'), (24, 'Carlo'), (25, 'Chicago'), (26, 'Claude'), (27, 'Come'), (28, 'Croft'), (29, 'Destroyed'), (30, 'Devonshire'), (31, 'Don'), (32, 'Dubarry'), (33, 'Emperors'), (34, 'Florence'), (35, 'For'), (36, 'Gallery'), (37, 'Gideon'), (38, 'Gisburn'), (39, 'Gisburns'), (40, 'Grafton'), (41, 'Greek'), (42, 'Grindle'), (43, 'Grindles'), (44, 'HAD'), (45, 'Had'), (46, 'Hang'), (47, 'Has'), (48, 'He'), (49, 'Her'), (50, 'Hermia'), (51, 'His'), (52, 'How'), (53, 'I'), (54, 'If'), (55, 'In'), (56, 'It'), (57, 'Jack'), (58, 'Jove'), (59, 'Just'), (60, 'Lord'), (61, 'Made'), (62, 'Miss'), (63, 'Money'), (64, 'Monte'), (65, 'Moon-dancers'), (66, 'Mr'), (67, 'Mrs'), (68, 'My'), (69, 'Never'), (70, 'No'), (71, 

In [35]:
for i,item in enumerate(vocab.items()):
    if i<50:
        print(item)

(0, '!')
(1, '"')
(2, "'")
(3, '(')
(4, ')')
(5, ',')
(6, '--')
(7, '.')
(8, ':')
(9, ';')
(10, '?')
(11, 'A')
(12, 'Ah')
(13, 'Among')
(14, 'And')
(15, 'Are')
(16, 'Arrt')
(17, 'As')
(18, 'At')
(19, 'Be')
(20, 'Begin')
(21, 'Burlington')
(22, 'But')
(23, 'By')
(24, 'Carlo')
(25, 'Chicago')
(26, 'Claude')
(27, 'Come')
(28, 'Croft')
(29, 'Destroyed')
(30, 'Devonshire')
(31, 'Don')
(32, 'Dubarry')
(33, 'Emperors')
(34, 'Florence')
(35, 'For')
(36, 'Gallery')
(37, 'Gideon')
(38, 'Gisburn')
(39, 'Gisburns')
(40, 'Grafton')
(41, 'Greek')
(42, 'Grindle')
(43, 'Grindles')
(44, 'HAD')
(45, 'Had')
(46, 'Hang')
(47, 'Has')
(48, 'He')
(49, 'Her')
